In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.isotonic import IsotonicRegression
from ideas import intersection_over_union
from ideas import bb_intersection_over_union
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, StratifiedKFold

In [59]:
columns = ['Xmin','Ymin','Xmax','Ymax']
box_columns = columns + ['W', 'L']
box_columns_gd = list(map(lambda x: x + '_gd', box_columns))
box_columns_fgd = list(map(lambda x: x + '_fgd', box_columns))
box_columns_iou = list(map(lambda x: x + '_iou', box_columns))

center_columns = ['XcenterDelta', 'YcenterDelta']
# ['XcenterDelta', 'YcenterDelta']
# 'Xratio', 'Yratio'
# 'box_iou', 'iou'
columns_ratio = ['iou']
# user_columns = ['XminError', 'XmaxError', 'YminError', 'YmaxError']
# ['XError', 'YError']
user_columns = []

box_columns_all = box_columns + box_columns_gd + center_columns + user_columns
box_columns_true = ['Xmin_true', 'Ymin_true', 'Xmax_true', 'Ymax_true']
box_columns_id = columns + ['itemId']

def lw(df):
    df['L'] = df['Xmax'] - df['Xmin']
    df['W'] = df['Ymax'] - df['Ymin']
    return df

In [60]:
te_d = lw(pd.read_csv('test_data_full.csv'))
t_d = lw(pd.read_csv('train_data_full.csv'))
t_a = pd.read_csv('train_answers.csv')
t_d_a = t_d.merge(t_a, on="itemId")
ids = list(set(te_d['itemId']))
scaler = MinMaxScaler()
scaler_fit_d = t_d_a[box_columns_all].values
scaler_fit_d = np.append(scaler_fit_d, te_d[box_columns_all].values, axis = 0)
scaler.fit(scaler_fit_d)
t_d_a[box_columns_all] = scaler.transform(t_d_a[box_columns_all])
te_d[box_columns_all] = scaler.transform(te_d[box_columns_all])

In [5]:
def my_custom_loss_func(y_true, y_pred):
    iou = np.zeros(len(y_true))
    for i in range(0, len(y_pred)):
        iou[i] = bb_intersection_over_union(y_true.iloc[i].values, y_pred[i])                          
    res = iou.mean()
    print(res)        
    return res

In [61]:
# for uid in uids:
X = t_d_a[box_columns_all + columns_ratio]
y = t_d_a[box_columns_true]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
# clf = linear_model.MultiTaskLassoCV(alphas=(0.1, 1, 10), eps=0.0001, cv=4)
# clf = MultiOutputRegressor(linear_model.LogisticRegression(solver='lbfgs', 
#                                                            multi_class='auto',
#                                                            max_iter=1000))
# clf = linear_model.ElasticNet(alpha=0.0005)
clf = linear_model.RidgeCV(alphas=(0.1, 1, 10), 
                           cv=4, scoring=make_scorer(my_custom_loss_func))
# clf = linear_model.LassoLarsCV()
clf.fit(X_train, y_train)

0.5563491541822062
0.5627708515929811
0.5555695975418264
0.5482659099521742
0.555492473960531
0.562162297941305
0.5547488067835394
0.5478176602962616
0.5410576800137021
0.5497106630535072
0.5417544805995556
0.5344336755758681


RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=4, fit_intercept=True,
        gcv_mode=None, normalize=False,
        scoring=make_scorer(my_custom_loss_func), store_cv_values=False)

In [62]:
y_test_pred = clf.predict(X_test)
iou = np.zeros(len(y_test_pred))
for i in range(0, len(y_test_pred)):
    iou[i] = bb_intersection_over_union(y_test_pred[i], y_test.iloc[i].values)
print(iou.mean())    

0.5558454440521365


In [16]:
X_submit = te_d[box_columns_all + columns_ratio]
y_submit = clf.predict(X_submit)
answers = pd.DataFrame(np.zeros((len(te_d), 5), dtype=int), index=te_d.index, columns=box_columns_id)
y = 0
for i,r in te_d.iterrows():        
    for ci,cv in enumerate(columns):
        answers.iloc[i][cv] = y_submit[y,ci] 
    answers.iloc[i]['itemId'] = r['itemId']
    y = y + 1
answers.groupby("itemId").mean().to_csv('submit2204.csv', header=False)